## Extracting Data from a CSV or XLSX file
Data Cleaning and Comparisons in Python, by Nickie Burns

In [1]:
import pandas as pd 
import numpy as np
from difflib import SequenceMatcher           
#pd.set_option('display.expand_frame_repr', False) 
pd.set_option('display.max_colwidth', -1)

# 0. Definitions, Background, etc.
* Using Python's Pandas library, datasets in various forms can be read in to form a Pandas Dataframe and then easily manipulated, cleaned, etc.
* A Pandas dataframe is very similar to a table, like those in Excel or Word.
* A row of a dataframe is a Pandas Series, which is like list with explicit indeces, and can easily be converted into a Python list.

In [2]:
regions_file = "Counties and Regions.csv" # uses pd.read_csv(filename)
house_file = "Representatives 4_22_2020.csv" # uses pd.read_csv (filename)
ia_ga_file = "NC_Gen_Assembly 05_07_2020.xlsx" # uses pd.read_excel(filename)
senate_file = "Senate 4_22_2020.csv" # uses pd.read_csv(filename)

# 1. NC County & Regions Data
* Pandas Dataframe creation from csv
* selecting column values based on other column values in same row
* converting a Pandas Series into a list

##### dataframe --> regions_df

In [3]:
regions_df = pd.read_csv(regions_file)
# regions_df

In [4]:
County = input("County:")

County:Wake


In [5]:
region = regions_df[regions_df["County"] == County]
region

,County,Region
62,Wake,Triangle


##### county and regions lists --> nc_county & nc_region

In [6]:
nc_county = regions_df['County'].tolist()
nc_region = regions_df['Region'].tolist()

In [7]:
#for c in nc_county: print(c)

In [8]:
#for r in nc_region: print(r)

# 2. SENATE DATA


##### initial dataframe --> senate_df 
learn how to read in csv files that have at least one row that is longer than the length of the csv header row 

In [9]:
#### nc senate dataframe ####

# fill the header row (names) of the dataframe because header row has to be the same size as the longest row
header = ['Party', 'District', 'Member'] 
for i in range(5):
    
    # this loops to create header like: cty0, cty1, cty2 ...cty4 (0-4, so 5 columns total, i.e. range(5))
    name = 'cty' + str(i)
    header.append(name)
    
# skip over the first row - which was the original row with skiprows=1
senate_df = pd.read_csv(senate_file, skiprows=1, names=header)
senate_df = senate_df.replace(np.nan, '', regex=True)
#senate_df

##### cleaned dataframe --> senate_df 
learn how to <br>(1) combine qualitative data of all rows in two columns to form a new column <br>(2) split qualitative data of all rows in one column into two or more columns <br>(3) filter data based on column values

In [10]:
# official house members list
s_member = senate_df["Member"].tolist()

# needed because some values of the counties column are suffixes
s_county = senate_df['cty0'].tolist()

# build suffixes list with nc county list
s_suffix = list(cty for cty in s_county if cty not in nc_county)

# filter out suffixes and concat with member
num_rows = senate_df.shape[0]
for i in range(num_rows):
    if s_county[i] in s_suffix:
        s_member[i] += s_county[i]
        s_county[i] = " "
        
# replace original members column with cleaned members (use a different name here)
senate_df.drop(['Member'], axis=1)
senate_df['Members'] = s_member

#split all non-name items from the member column into its own column 'Notes'
senate_df[["Member", "Notes", "Notes1"]] = senate_df.Members.apply(lambda x: pd.Series(str(x).split("(")))

# filter out all resigned and deceased members
senate_df["cty"] = s_county
senate_df = senate_df[~senate_df["Notes1"].str.contains("Resign", na=False)]
senate_df = senate_df[~senate_df["Notes"].str.contains("Resign", na=False)]
senate_df = senate_df[~senate_df["Notes1"].str.contains("Deceas", na=False)]
senate_df = senate_df[~senate_df["Notes"].str.contains("Deceas", na=False)]

# reorder columns, and don't include cty0
senate_df = senate_df[["Party", "District", "Member", "cty", "cty1", "cty2", "cty3", "cty4", "Notes", "Notes1"]]
senate_df = senate_df.replace(np.nan, '', regex=True)
#senate_df

##### senate members list --> s_members 
learn how to <br>(1) convert a pandas dataframe row, which is a Pandas Series, into a list <br>(2) Strip heading and trailing white space from the values of a list

In [11]:
s_members_init = senate_df["Member"].tolist()
s_members = [m.strip() for m in s_members_init]
#for m in s_members: print(m)

# 3. HOUSE DATA

##### initial dataframe --> house_df

In [12]:
house_df = pd.read_csv(house_file)
house_df = house_df.replace(np.nan, '', regex=True)
#house_df

##### cleaned dataframe --> house_df

In [13]:
# official house members list
h_member = house_df["Member"].tolist()

# needed because some values of the counties column are suffixes
h_county = house_df['Counties Represented'].tolist()

# build suffixes list with nc county list
nc_county = regions_df["County"].tolist()
h_suffix = list(cty for cty in h_county if cty not in nc_county)

# filter out suffixes and concat with member
num_rows = house_df.shape[0]
for i in range(num_rows):
    if h_county[i] in h_suffix:
        h_member[i] += h_county[i]
        h_county[i] = " "

# replace original members column with cleaned members (use a different name here)
house_df.drop(['Member'], axis=1)
house_df['Members'] = h_member

# split all non-name items from the member column into its own column 'Notes'
house_df[['Member','Notes']] = house_df.Members.apply( 
   lambda x: pd.Series(str(x).split("(")))

# replace original countries column with cleaned countries
house_df['Counties'] = h_county
house_df = house_df[~house_df["Notes"].str.contains("Resign", na=False)]
house_df = house_df[~house_df["Notes"].str.contains("Deceased", na=False)]

# reorder the columns
house_df = house_df[["Party", "District", "Member", "Counties", "Unnamed: 4", "Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Notes"]]

# store the members column in a list
h_members = house_df["Member"].tolist()

# un4 = house_df['Unnamed: 4'].tolist()
# un5 = house_df['Unnamed: 5'].tolist()
# un6 = house_df['Unnamed: 6'].tolist()
# un7 = house_df['Unnamed: 7'].tolist()
# un8 = house_df['Unnamed: 8'].tolist()
house_df = house_df.replace(np.nan, '', regex=True)
#house_df

##### house members list --> h_members

In [14]:
h_members_init = house_df["Member"].tolist()
h_members = [m.strip() for m in h_members_init]
# for m in h_members: print(m)

# 4. InterAction DATA

##### initial dataframe --> ia_df 

In [15]:
ia_df = pd.read_excel(ia_ga_file)
ia_df = ia_df.replace(np.nan, '', regex=True)
#ia_df

##### cleaned dataframe --> ia_df

In [16]:
first = ia_df["Goes_By"].tolist()
middle = ia_df["Middle_Name"].tolist()
last = ia_df["Last_Name"].tolist()
suf = ia_df["Suffix"].tolist()

members = list()
for i in range(len(first)):
    if middle[i] != "":
        member = first[i] + " " + middle[i] + " " + last[i]
    else:
        member = first[i] + " " + last[i]
    if suf[i] != "":
        member += " " + suf[i]
    members.append(member)

ia_df["Member"] = members

#ia_df

##### IA senate & house member lists --> s_members_ia & h_members_ia

In [17]:
h_df = ia_df[ia_df.Company_Name.str.contains('Rep')]
s_df = ia_df[ia_df.Company_Name.str.contains('Sen')]

h_df = h_df.replace(np.nan, '', regex=True)
s_df = s_df.replace(np.nan, '', regex=True)

h_members_ia = h_df["Member"].tolist()
s_members_ia = s_df["Member"].tolist()

In [18]:
# for m in s_members_ia: print(m)

In [19]:
# for m in h_members_ia: print(m)

# 5. FINAL COMPARISONS

##### members

In [20]:
# this dictionary pairs similar names 

def create_dicts_to_replace_member_values(list_ia, list_nc):
    
    add = [x for x in list_ia if x not in list_nc]
    delete = [y for y in list_nc if y not in list_ia]

    dictionary = dict()

    for i in range(len(add)):
        for d in delete:

            # SequenceMatcher determines the similarity of two strings
            if SequenceMatcher(None, add[i], d).ratio() > .6:

                # which are stored here if they are similar 
                dictionary.update( { add[i] : d})

    for i in range(len(delete)):
        for a in add:

            # SequenceMatcher determines the similarity of two strings
            if SequenceMatcher(None, delete[i], a).ratio() > .6:

                # which are stored here if they are similar
                dictionary.update( { delete[i] : a})
    
    return add, delete, dictionary

In [21]:
s_add, s_delete, s_dict = create_dicts_to_replace_member_values(s_members_ia, s_members)
h_add, h_delete, h_dict = create_dicts_to_replace_member_values(h_members_ia, h_members)

In [22]:
#s_dict

In [23]:
#h_dict

In [24]:
for s in s_delete:
    if s not in s_dict:
        print("Check whether Senator", s, "needs to be deleted from the IA General Assembly list")
        print (" ")
for s in s_add:
    if s not in s_dict:
        print("Check whether Senator", s, "needs to be added to the IA General Assembly list.")
        print(" ")

In [25]:
for h in h_delete:
    if h not in h_dict:
        print("Check whether Representative", h, "needs to be deleted from the IA General Assembly list")
        print (" ")

for h in h_add:
    if h not in h_dict:
        print("Check whether Representative", h, "needs to be added to the IA General Assembly list.")
        print(" ")

##### district & political party

In [26]:
house_party_dist = house_df[["Member", "Party", "District"]].reset_index(drop=True)
senate_party_dist = senate_df[["Member", "Party", "District"]].reset_index(drop=True)

In [27]:
h_party_dist_ia = h_df[["Member", "Party_Affiliation", "NC_House_District"]].reset_index(drop=True)
s_party_dist_ia = s_df[["Member", "Party_Affiliation", "NC_Senate_District"]].reset_index(drop=True)

parties = h_df["Party_Affiliation"].tolist()
length = len(parties)
h_party_list = list()
for i in range(length):
    h_party_list.append(parties[i][0])
h_party_dist_ia["Party"] = h_party_list
h_party_dist_ia = h_party_dist_ia[["Member", "Party", "NC_House_District"]].reset_index(drop=True)
h_party_dist_ia

,Member,Party,NC_House_District
0,Jay Adams,R,96.0
1,Gale Adcock RN,D,41.0
2,John Ager,D,115.0
3,Kelly M. Alexander Jr.,D,107.0
4,Vernetta Alston,D,29.0
...,...,...,...
115,Donna McDowell White,R,26.0
116,Shelly Willingham,D,23.0
117,Michael H. Wray,D,27.0
118,Larry Yarborough,R,2.0


In [28]:
house_party_dist.head()

,Member,Party,District
0,Jay Adams,R,96
1,Gale Adcock,D,41
2,John Ager,D,115
3,Kelly M. Alexander Jr.,D,107
4,Vernetta Alston,D,29


In [29]:
h_party_dist_ia.head()

,Member,Party,NC_House_District
0,Jay Adams,R,96.0
1,Gale Adcock RN,D,41.0
2,John Ager,D,115.0
3,Kelly M. Alexander Jr.,D,107.0
4,Vernetta Alston,D,29.0


In [30]:
h_party_dist_ia['political party correct?'] = np.where(house_party_dist['Party'] == h_party_dist_ia['Party'], 'True', 'False')
h_party_dist_ia['district party correct?'] = np.where(house_party_dist['District'] == h_party_dist_ia['NC_House_District'], 'True', 'False')

In [31]:
h_party_dist_ia

,Member,Party,NC_House_District,political party correct?,district party correct?
0,Jay Adams,R,96.0,True,True
1,Gale Adcock RN,D,41.0,True,True
2,John Ager,D,115.0,True,True
3,Kelly M. Alexander Jr.,D,107.0,True,True
4,Vernetta Alston,D,29.0,True,True
...,...,...,...,...,...
115,Donna McDowell White,R,26.0,True,True
116,Shelly Willingham,D,23.0,True,True
117,Michael H. Wray,D,27.0,True,True
118,Larry Yarborough,R,2.0,True,True


In [32]:
h_party_dist_ia_fix = h_party_dist_ia[h_party_dist_ia['political party correct?'] == False]
h_party_dist_ia_fix2 = h_party_dist_ia_fix[h_party_dist_ia_fix['district party correct?'] == False]
h_party_dist_ia_fix

,Member,Party,NC_House_District,political party correct?,district party correct?
